In [9]:
# Cell 1: imports
import pandas as pd

# Cell 2: load data
df = pd.read_excel("data/Online Retail.xlsx")
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
# Cell 3: info & nulls
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [11]:
# Cell 4: detect cancelled invoices & duplicates
print("Cancelled:", df[df["InvoiceNo"].astype(str).str.startswith("C")].shape[0])
print("Duplicates:", df.duplicated().sum())


Cancelled: 9288
Duplicates: 5268


In [12]:
# Cell 5: preliminary cleaning
df_clean = df.drop_duplicates()
df_clean = df_clean[~df_clean["InvoiceNo"].astype(str).str.startswith("C")]


In [13]:
#Fecha referencia 

snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

In [14]:
# Sólo clientes con ID (sin nulls)
df_customers = df_clean.dropna(subset=['CustomerID'])

# Cálculo de métricas RFM
rfm = df_customers.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                 # Frequency
    'UnitPrice': lambda x: (x * df_customers.loc[x.index, 'Quantity']).sum()  # Monetary
})

# Renombrar columnas
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'UnitPrice': 'MonetaryValue'
}, inplace=True)

# Revisar resultado
print(rfm.head())


            Recency  Frequency  MonetaryValue
CustomerID                                   
12346.0         326          1       77183.60
12347.0           2          7        4310.00
12348.0          75          4        1797.24
12349.0          19          1        1757.55
12350.0         310          1         334.40


In [ ]:
#    Cada valor único de Frequency recibe un entero consecutivo.
rfm['Freq_Rank'] = rfm['Frequency'].rank(method='dense')

# 2. Verifica los primeros valores
print(rfm[['Frequency', 'Freq_Rank']].drop_duplicates().sort_values('Frequency').head(10))

# Crear funciones de scoring basadas en quintiles
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['F_Score'] = pd.qcut(rfm['Freq_Rank'], 5, labels=[1,2,3,4,5], duplicates='drop').astype(int)
rfm['M_Score'] = pd.qcut(rfm['MonetaryValue'], 5, labels=[1,2,3,4,5]).astype(int)

# Concatenar en un solo código RFM
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) \
                    + rfm['F_Score'].astype(str) \
                    + rfm['M_Score'].astype(str)

# Añadir RFM Score total si quieres (opcional)
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].sum(axis=1)

# Verifica
print(rfm[['Recency','Frequency','MonetaryValue','RFM_Segment','RFM_Score']].head())


            Frequency  Freq_Rank
CustomerID                      
12346.0             1        1.0
12358.0             2        2.0
12356.0             3        3.0
12348.0             4        4.0
12381.0             5        5.0
12388.0             6        6.0
12347.0             7        7.0
12352.0             8        8.0
12417.0             9        9.0
12362.0            10       10.0


ValueError: Bin labels must be one fewer than the number of bin edges

In [ ]:
# Exporta a CSV para tu dashboard y README
rfm.to_csv('data/rfm_segments.csv', index=True)